# Fast text - TripleM

## Install requirements

## Load Libraries

In [ ]:
import fasttext.util
import zipfile
import gensim
import numpy as np
import json
import random
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralCoclustering
from scipy.stats import gaussian_kde
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, SpectralClustering, DBSCAN

## Settings

In [ ]:
path = "/home/matej/Documents/FRI/NLP/NLP-tripleM/src"

## Load Fasttext model

In [ ]:
path_to_fast_text_model = f"{path}/pretrained models/Fasttext/cc.sl.300.bin"
ft = fasttext.load_model(path_to_fast_text_model)

## Load data

In [ ]:
with open('/mydrive/data.json') as json_file:
    data = json.load(json_file)
with open('/mydrive/data_lema.json') as json_file:
    data_lema = json.load(json_file)
with open('/mydrive/data_pos.json') as json_file:
    data_len = json.load(json_file)

words = np.load("/mydrive/words.npy")
words

array(['leto', 'dan', 'konec', 'svet', 'stran', 'mesto', 'šola', 'ura',
       'beseda', 'pot', 'red', 'zakon', 'zadeva', 'srce', 'tema',
       'resnica', 'moški', 'vloga', 'kraj', 'stanje', 'škoda', 'film',
       'večer', 'vrh', 'jutro', 'kazen', 'oblast', 'račun', 'novica',
       'milijon', 'par', 'krog', 'tip', 'punca', 'sila', 'vir', 'las',
       'akcija', 'meter', 'prst', 'kri', 'stik', 'grad', 'znak', 'lik',
       'direktor', 'vodja', 'raven', 'kolo', 'rob', 'gost', 'duh',
       'praznik', 'vest', 'korist', 'vedenje', 'tek', 'kup', 'otok',
       'razstava', 'bitje', 'motor', 'karta', 'nevarnost', 'hitrost',
       'kos', 'zob', 'stroj', 'kamen', 'župan', 'šef', 'vrtec', 'kot',
       'deček', 'avgust', 'tok', 'jezero', 'klop', 'čelo', 'hip', 'kupec',
       'pojav', 'čaj', 'postava', 'dolg', 'standard', 'jesen', 'rak',
       'grob', 'plus', 'les', 'vez', 'polica', 'minus', 'plan', 'posoda',
       'restavracija', 'jok', 'krilo', 'sol', 'rod', 'stres', 'trditev',
       'f

## Prepare sentences

In [ ]:
min_number_of_words = 8
min_neighbor_distance = 6

In [ ]:
number_of_sentences_to_use = 3000
for keyword in ['golf']:
    all_sentences2=data[keyword][:min(number_of_sentences_to_use, len(data[keyword]))]
    all_sentences_lema2=data_lema[keyword][:min(number_of_sentences_to_use, len(data[keyword]))]

    all_sentences = []
    all_sentences_lema = []
    for sentence, sentence_lema in zip(all_sentences2, all_sentences_lema2):
      if len(sentence) >= min_number_of_words:

        if sentence_lema not in all_sentences_lema:
          all_sentences.append(sentence)
          all_sentences_lema.append(sentence_lema)


    
    #Remove duplicate sentences if any
    #all_sentences=remove_diplicate_sentences(all_sentences)
    #all_sentences_lema=remove_diplicate_sentences(all_sentences_lema)

    print(len(all_sentences))
    
    all_embeddings=np.zeros((len(all_sentences),300))
    #all_embeddings = []
    
    for i in range(len(all_sentences)): #iterate through the sentences for the given keyword

        keyword_position = all_sentences_lema[i].index(keyword)
        #all_sentences[i].pop(keyword_position)
        
        centroid=np.zeros(300)
        words_added=0
        
        for word_index, word in enumerate(all_sentences[i]): #iterate through the words in the sentence
            
              if word_index!=keyword_position and abs(word_index - keyword_position) < min_neighbor_distance:
                  centroid+=ft.get_word_vector(word.lower())
                  words_added+=1
                
        if words_added==0: #there are wierd sentences where only the keyword is the whole sentence
            continue
            
        centroid/=words_added
        all_embeddings[i,:]=centroid
          
    
    break #this break means that we terminate on the first word
    
#~np.all(all_embeddings == 0, axis=1) this checks if there are any only-zero rows meaning that we did not have any word embedding for that sentence

2064


In [ ]:
#Some utility functions which are used
def cos_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def remove_diplicate_sentences(a):
    b_set = set(map(tuple,a))  #need to convert the inner lists to tuples so they are hashable
    b = list(map(list,b_set)) #Now convert tuples back into lists (maybe unnecessary?)
    return b

In [ ]:
distances={}
for i in range(all_embeddings.shape[0]):
    for j in range(i+1,all_embeddings.shape[0],1):
        distances[str(i)+'-'+str(j)] = cos_similarity(all_embeddings[i,:], all_embeddings[j,:])

In [ ]:
distances = dict(sorted(distances.items(), key=lambda x:x[1]))

## Analyze worst 50 distances

In [ ]:
distances_keys = list(distances.keys())
already_were = [] #to je zato ker se stavke ponavljajo in ne zanimajo nas iste kombinacije znova
count=0
for key in distances_keys[0:20]:
    print(distances[key])
    first, second = map(int, key.split('-'))
    first_sent, second_sent = ' '.join(all_sentences[first]), ' '.join(all_sentences[second])
    
    if first_sent+second_sent in already_were:
        continue
    
    count+=1
    already_were.append(first_sent+second_sent)
    print(first_sent+'\n'+second_sent)
    print('-----')
    if count==20:
        break

-0.45298597292820186
V noči na torek so z Ljubljanske ulice v Grosupljem ukradli cordobinega bratranca prav tako modrega golfa 1.9 TDI
GOLF D II 5V prodam # 031 791 479
-----
-0.4394853786941152
Omeniti velja tudi golfsko igrišče Penha Longa Golf Club
GOLF D II 5V prodam # 031 791 479
-----
-0.42207533231170763
GOLF Odprto prvenstvo Slovenije nadaljevanje danes in jutri ob 8. uri na igrišču na Bledu
GOLF D II 5V prodam # 031 791 479
-----
-0.4189564452759347
GOLF 1.9 TDI letnik 1994 100.000 km 5V rdeč euro kljuka CZ prodam # 031 382 088
GOLF D II 5V prodam # 031 791 479
-----
-0.4141880351830505
Tenis atletika košarka nogomet rokomet golf zelo specifični športi
Prodam Golf JX l. 88 reg. do 8 99
-----
-0.4137894263317615
GOLF D II 5V prodam # 031 791 479
GOLF 1.9 TDI VARIANT LET 2000 MET SREBRN KLIMA ABS 4X AIR BAG OHRANJEN 2.640.000 SIT AVTO LESCE D.
-----
-0.4078497150357563
Člani Golf kluba grad Otočec Novo mesto plačajo samo najem igralne površine
GOLF D II 5V prodam # 031 791 479
-

## Analyze best 50 distances

In [ ]:
distances_keys = list(distances.keys())
already_were = [] #to je zato ker se stavke ponavljajo in ne zanimajo nas iste kombinacije znova
count=0
for key in distances_keys[-20:]:
    print(distances[key])
    first, second = map(int, key.split('-'))
    first_sent, second_sent = ' '.join(all_sentences[first]), ' '.join(all_sentences[second])
    if first_sent+second_sent in already_were:
        continue
    
    count+=1
    already_were.append(first_sent+second_sent)
    print(first_sent+'\n'+second_sent)
    print('-----')
    if count==20:
        break

0.967584459508883
Teri in njena hči sta že nekajkrat tudi bivali v rezidenci Bushevih v Mainu igralka je družini spekla ovsene piškote šli so na ribolov in igrali golf
Golfi passati in seati so na Primorskem in Notranjskem izginjali od novembra lani do januarja letos ko so tatovom na prste stopili kriminalisti
-----
0.9700030372688244
Royal and Ancient Golf Club of St. Andrews seveda še vedno ostaja klub odprt le moškim kar je v napadu feminizma tudi super
Če je igralec v dvomu da bi z uporabo določenega pripomočka kršil 14.3 pravilo se mora posvetovati s klubom The Royal and Ancient Golf Club of St. Andrews
-----
0.9737352934198731
Če je igralec v dvomu da bi z uporabo določenega pripomočka kršil 14.3 pravilo se mora posvetovati s klubom The Royal and Ancient Golf Club of St. Andrews
Izdelovalec lahko pošlje klubu The Royal and Ancient Golf Club of St. Andrews vzorec pripomočka ki ga namerava izdelovati s prošnjo za presojo njegove skladnosti s 14.3 pravilom če bi ga igralec uporabil 

## Construct similarity distance matrix

In [ ]:
#print(all_embeddings.shape)
distance_matrix=(pairwise_distances(all_embeddings,metric="cosine"))
similarity_matrix=1-distance_matrix


#density plot
similarity_matrix_flatten = similarity_matrix.reshape(len(similarity_matrix)**2)
density = gaussian_kde(similarity_matrix_flatten)
density.covariance_factor = lambda : .5
len(similarity_matrix_flatten)
#plt.plot(similarity_matrix_flatten, density(similarity_matrix_flatten))
#plt.show()


4260096

In [ ]:
np.max(distance_matrix)

1.4529859729282022

In [ ]:
np.min(similarity_matrix)

-0.4529859729282022

In [ ]:
best_distances_indices=[]

best_scores_indices=[]
for i in list(distances.keys()):
    first, second = i.split('-')
    if distances[i]>0.9:
        best_scores_indices.append(first)
        best_scores_indices.append(second)


best_scores_indices=(np.unique(best_scores_indices))
best_scores_indices = best_scores_indices.astype(int)
all_embeddings_selected = all_embeddings[best_scores_indices, :]

distance_matrix=(pairwise_distances(all_embeddings_selected,metric="cosine"))

In [ ]:
all_sentences_selected = []
for i in best_scores_indices:
    all_sentences_selected.append(all_sentences[i])

## Clustering

In [ ]:
clusters = AgglomerativeClustering(affinity='precomputed', linkage='single', n_clusters=2).fit(distance_matrix)
print(Counter(clusters.labels_))
clusters = AgglomerativeClustering(affinity='precomputed', linkage='average', n_clusters=2).fit(distance_matrix)
print(Counter(clusters.labels_))
clusters = AgglomerativeClustering(affinity='precomputed', linkage='complete', n_clusters=2).fit(distance_matrix)
print(Counter(clusters.labels_))

Counter({0: 599, 1: 4})
Counter({0: 579, 1: 24})
Counter({0: 577, 1: 26})


In [ ]:
for i,label in enumerate(clusters.labels_):
    if label==1:
        print(i,':', ' '.join(all_sentences_selected[i]))

4 : 9.00 Golf evropska turneja 9.30 Reklamna oddaja 10.00 Confederations Cup 12.00 Reklamna oddaja 12.30 Confederations Cup 14.30 NHL Stanley Cup 15.15 Pago Pago 16.00 Xapatan 16.30 Takeshi's Castle 17.00 Pago Pago 17.45 Košarka 18.00 DSF novinarski center 18.30 DSF reportaža 19.00 Dvoboj DSF kviz 19.40 DSF novinarski center 20.00 U 21 22.00 DSF novinarski center 22.15 Confederations Cup 23.15 Touchdown Gatorade nogometni magazin 23.45 Golf evropska turneja 0.45 Extreme Situations 1.15 Reklamna oddaja 1.45 Reklamna oddaja
47 : 8.30 Jahanje pon. 9.30 Golf pon. 10.30 Motorsport pon. 11.00 Automagazin
51 : Članice Golf kluba Grad Otočec imajo svojo sekcijo in uspešno predstavljajo klub na medklubskih tekmovanjih v Sloveniji
87 : 8.00 Iz domače skrinje ponovitev 10.00 Evropski golf turnirji ponovitev 10.30 TV prodaja 11.30 Video strani 13.30 TV prodaja 14.30 Poslovne informacije 15.00 Iz domače skrinje ponovitev 16.30 Motor Show Report 17.00 Juke Box kontaktna oddaja 18.30 Avto šou 19.00 R

In [ ]:
spectral = SpectralClustering(3, affinity="precomputed").fit_predict(distance_matrix)
print(Counter(spectral))


Counter({2: 208, 0: 204, 1: 191})


In [ ]:
for i,label in enumerate(spectral):
    if label==2:
        print(i,':', ' '.join(all_sentences_selected[i]))

1 : Opoldne se z Aaronom odpeljeva na partijo golfa
2 : Vsaj če sodimo po novem Volkswagnovem bencinskem izdelku ki so mu nadeli ime TSI in je na voljo tudi v golfu
6 : Golf za telo in dušo Lilijana Kenda lastnica galerije Lala pravi da je golf odlična rekreacija
9 : No če ste mogoče pomislili da je česa manjkalo se motite vsega je bilo ravno prav in najbolj zagreti so se lahko poskusili še v pravem golfu
10 : Sonja pa tudi odvetnik Peter Čeferin so pokazali da palice za golf v rokah zagotovo niso držali prvič
11 : Odprto prvenstvo ZDA v golfu ob 102. izvedbi prvič na javnem igrišču
12 : V zadku je prtljažnik ki z najmanj 246 litri pri odprti in 315 litri pri zaprti strehi sprejme dve torbi z opremo za golf
13 : Dobri primeri so Andy Roddick Richard Gasquet in Tatiana Golovin v tenisu ali Lorena Ochoa Cristie Kerr José Maria Olazabal in Raphaël Jacquelin v golfu
15 : Za poznavalce golfa ki so na tem mestu verjetno postali že strašno radovedni naj povemo da so za razliko od zelenega pri

In [ ]:
dbscan = DBSCAN(metric='cosine', eps=0.4, min_samples=3).fit(all_embeddings_selected)  # you can change these parameters, given just for example 
labels = dbscan.labels_ # where X - is your matrix, where each row corresponds to one document (line) from the docs, you need to cluster 
#cluster_labels
no_clusters = len(np.unique(labels) )
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

print(Counter(labels))
#all_sentences[np.where(labels == 1)]
for i,label in enumerate(labels):
    if label==4:
        print(all_sentences_selected[i])

Estimated no. of clusters: 6
Estimated no. of noise points: 14
Counter({0: 568, -1: 14, 1: 6, 4: 6, 2: 5, 3: 4})
['GOLF', '1.4', 'CL', 'letnik', '1996', 'dobro', 'ohranjen', 'ugodno', 'prodam', '#', '0608', '43', '313']
['GOLF', '1.9', 'tdi', 'klimatronik', 'limuzina', 'letnik', '1998', 'prodam']
['GOLF', '1.4', 'CL', 'letnik', '1993', '42.000', 'km', '4', 'vrata', 'rdeč', 'ohranjen', 'prodam', '#', '068', '44', '784']
['GOLF', 'Variant', 'Basis', '1.9', 'TDi', '74kW', '4MOT', '3.991.890']
['GOLF', 'Variant', 'Basis', '1.9', 'TDi', '66kW', 'Avt.', '3.835.677']
['GOLF', '1.9', 'TDI', 'letnik', '1994', '100.000', 'km', '5V', 'rdeč', 'euro', 'kljuka', 'CZ', 'prodam', '#', '031', '382', '088']
